In [16]:
#Libraries used in the functions below
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import math

from jupyter_dash import JupyterDash
import os
try:
    os.environ.pop('http_proxy')
    os.environ.pop('https_proxy')
except KeyError:
    pass

# Import dash
import dash
from dash import Dash, html, dcc, Input, Output, dash_table, State

from dash_bootstrap_components.themes import BOOTSTRAP

# Import plotly
import plotly.graph_objs as go
import plotly.express as px
import seaborn as sns

In [2]:
#Datasets turned into Pandas DataFrames
dfC = pd.read_csv("dataset/courses.csv")
dfSA=pd.read_csv("dataset/studentAssessment.csv")
dfSI=pd.read_csv("dataset/studentInfo.csv")
dfSR=pd.read_csv("dataset/studentRegistration.csv")
dfVLE=pd.read_csv("dataset/vle.csv")
dfSVLE=pd.read_csv("dataset/studentVle.csv")
dfA=pd.read_csv("dataset/assessments.csv")

In [3]:
#A list of all courses
dfC['Course_Section'] = dfC[['code_module', 'code_presentation']].apply(lambda x: '-'.join(x), axis=1)
course_section=list(dfC['Course_Section'] )

In [4]:
#This function provides the roster of a specific course and section
dfSI['Course_Section'] = dfSI[['code_module', 'code_presentation']].apply(lambda x: '-'.join(x), axis=1)
def roster(course_section):
    dfClassRoster=dfSI[dfSI['Course_Section']==course_section]
    return list(dfClassRoster['id_student'])

In [7]:
dfA['Course_Section'] = dfA[['code_module', 'code_presentation']].apply(lambda x: '-'.join(x), axis=1)
resource_dict = dict(zip(dfVLE['id_site'], dfVLE['activity_type']))
def classCode(Module,Presentation):
    dfCode=dfSVLE[dfSVLE['code_module']==Module]
    dfCode=dfCode[dfCode['code_presentation']==Presentation]
    return dfCode

In [39]:
#external_stylesheets = [BOOTSTRAP] 
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server

# Create a unique list of code_module
available_indicators1 = dfC['Course_Section']

# Step 1
app.title = "OU Dashboard"
app.layout = html.Div([
                        html.H1("Instructor Interface/Dashboard",style={'textAlign':'center'}),
                        html.Hr(),
                        html.Div([
                            html.H5("Module-Presentation"),
                            dcc.Dropdown(
                                id='CourseSection',
                                options=[{'label': i, 'value': i} for i in available_indicators1],
                                value=available_indicators1[0]),
                             dcc.RadioItems(
                                ['Assessment', 'Resources'],
                                'Assessment',
                                id='Graph-Type',
                                inline=True),
                            dcc.Graph(id='Graph'),
                                dcc.Slider(
                                    0,
                                    200,
                                    step=None,
                                    id='Day',
                                    value=60),
                            dash_table.DataTable(
                                    style_data={'width': '30%'},
                                    style_cell={
                                        'minWidth': '90px', 'width': '90px', 'maxWidth': '90px',
                                        'overflow': 'hidden',
                                        'textOverflow': 'ellipsis'},
                                id='Summary',
                                columns=[{"name": i, "id": i} for i in dfC.columns],
                                data = dfC.to_dict('records'),
                                filter_action="native",
                                sort_action="native",
                                sort_mode="multi",
                                page_size = 1 
            )
                        ], style={'width': '45%', 'display': 'inline-block', 'verticalAlign':'top'}),
                        html.Div([], style={'width':'2%', 'display':'inline-block'}),
                        html.Div([
                             html.H6("Sort by:"),
                             dcc.RadioItems(
                                ['Numerical', 'Failure Rate'],
                                 value='Numerical',
                                 id='Sort-By',
                            inline=True),
                            
                            dcc.Dropdown(
                                id='StudentID'),
                            
                            html.H6("Table: Selected Student Information"),
                            dash_table.DataTable(
                                id='StudentID Information',
                                columns=[{"name": i, "id": i} for i in dfC.columns],
                                data = dfC.to_dict('records'),
                                filter_action="native",
                                sort_action="native",
                                sort_mode="multi",
                                page_size = 10 
            ),
                            dcc.Textarea(
                                id='Recommendations',
                                value='Recommend actions would go here!',
                                style={
                                        'maxWidth': '100%', 
                                        'width': '100%',
                                        'height': 100
                                    },
    )

                        ], style={'width': '45%', 'display': 'inline-block'})
            ], style={'width':'90%', 'margin':'auto'})

        
@app.callback(
    Output('StudentID', 'options'),
    Input('CourseSection', 'value'))
def set_course_options(selected_course):
    return roster(selected_course)

@app.callback(
    Output('StudentID', 'value'),
    Input('StudentID', 'options'))
def set_id_value(available_options):
    return available_options[0]

@app.callback(
    Output('Recommendations','value'),
    Input('StudentID','value'),
    Input('CourseSection','value'),
    Input('Day','value')
)
def update_recommendation(id_student, course_section, day):
    return "Recommend actions would go here! \n Student: {0} \t Course: {1} \t Day: {2}".format(id_student, course_section, day)
    

@app.callback(
    Output('Graph', 'figure'),Input('CourseSection', 'value'),Input('StudentID', 'value'),
    Input('Graph-Type','value'),Input('Day','value'))

def update_figure(CourseSection1, StudentID1,type1,day1): 
    if type1=='Assessment':
        dfA1=dfA[dfA['date']<=day1]
        dfSA1=dfSA[dfSA['date_submitted']<=day1]
        dfCourse_Assessments=dfA1[dfA1['Course_Section']==CourseSection1]
        assessment=list(dfCourse_Assessments['id_assessment'])
        dictMean={}
        for assess in assessment:
            dfSAMean=dfSA1[dfSA1['id_assessment']==assess]
            dictMean[assess]=np.mean(dfSAMean['score'])
        dfStudentScore=dfSA1[dfSA1['id_student']==StudentID1]
        dfStudentScore=dfStudentScore[['id_assessment','score']]
        dfMean = pd.DataFrame(list(dictMean.items()), columns = ['id_assessment','score'])
        dfMean['type']='Mean'
        dfStudentScore['type']='Student'
        dfTogether=pd.concat([dfStudentScore, dfMean], ignore_index=True,axis=0)
        dfTogether['id_assessment'] = dfTogether.id_assessment.astype(str)
        dfTogether=dfTogether.dropna()
        figure = px.bar(dfTogether, x="id_assessment", y="score", color="type", barmode="group",
                title="Comparison of Students Scores and Class Averages")
        return figure
    else:
        dfSVLE1=dfSVLE[dfSVLE['date']<=day1]
        dfCode=dfSVLE1[dfSVLE1['code_module']==CourseSection1[:3]]
        dfCode=dfCode[dfCode['code_presentation']==CourseSection1[4:]]
        student_count=len(dfCode['id_student'].unique())
        dfCode=dfCode.replace({"id_site": resource_dict})
        dfResource=dfCode.groupby(['id_site']).sum()
        dfResource['sum_click']=dfResource['sum_click']/student_count
        dfResource=dfResource.reset_index()
        dfStudent=dfCode[dfCode['id_student']==StudentID1]
        dfStudent=dfStudent.groupby(['id_site']).sum()
        dfStudent=dfStudent.reset_index()
        dfStudent['type']='Student'
        dfResource['type']='Mean'
        dfTogether1=pd.concat([dfStudent, dfResource], ignore_index=True,axis=0)
        figure2 = px.bar(dfTogether1, x="id_site", y="sum_click", color="type", barmode="group",
            title="Comparison of Students Resource Usage and Class Averages")
        return figure2

# Run the app
app.run_server(mode="external", port = 8110, debug=False)